In [1]:
import pandas as pd
import numpy as np

# Read rt_leavetimes_DB_2018 file

This file is read to get the total number of stops for each trip in the trip file

# Explain the meaning of each feature

- DATASOURCE: Bus operator code
- DAYOFSERVICE: date of service of the bus
- TRIPID: Trip ID to check if it is the same trip that needs to be combined with DAYOFSERVICE
- LINEID: Route ID
- ROUTEID: Route ID
- DIRECTION indicates the direction of the bus form (forward or reverse)
- PLANNEDTIME_ARR Planned trip arrival time
- PLANNEDTIME_DEP Planned trip departure time
- ACTUALTIME_ARR Actual trip arrival time
- ACTUALTIME_DEP Actual trip departure time
- BASIN Basin code
- TENDERLOT Tender lot
- SUPPRESSED Whether suppressed at this station (0 for arrival, 1 for suppressed)
- JUSTIFICATIONID fault code
- LASTUPDATE Time of last update
- NOTE Note

### Read the file

In [2]:
chunks=10**6

In [3]:
i=0
df_le={}
for chunk in pd.read_csv('/tmp/data/rt_leavetimes_DB_2018.txt', chunksize=chunks,delimiter=';',keep_default_na=True):
    try:
        print(i)
        df_le[i]=chunk 
        i=i+1   
    except Exception as e:
        print(e)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [4]:
df_le[0].head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
1,DB,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
2,DB,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
3,DB,01-JAN-18 00:00:00,5966888,12,119,58801,58801,58771,58771,2693284,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
4,DB,01-JAN-18 00:00:00,5965960,12,119,56401,56401,56309,56323,2693209,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN


In [5]:
df_le[0].shape

(1000000, 18)

In [6]:
for i in range(0,117):
    if i%10==0:
        print(i)
    df_le[i].drop(labels=['DATASOURCE','STOPPOINTID','PLANNEDTIME_ARR','PLANNEDTIME_DEP','ACTUALTIME_ARR','ACTUALTIME_DEP','VEHICLEID','PASSENGERS','PASSENGERSOUT','DISTANCE','SUPPRESSED','JUSTIFICATIONID','LASTUPDATE','PASSENGERSIN','NOTE'],axis=1,inplace=True)  

0
10
20
30
40
50
60
70
80
90
100
110


In [7]:
chunks=[]
for i in range(0,117):
    if i%10==0:
        print(i)
    chunks.append(df_le[i])

0
10
20
30
40
50
60
70
80
90
100
110


In [8]:
df_le={}

In [9]:
df_pro = pd.concat(chunks, axis=0, ignore_index=True)

In [10]:
chunks=[]

### Get the required fields, DAYOFSERVICE, TRIPID and trip dataset correspondence

In [11]:
df_pro.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER
0,01-JAN-18 00:00:00,5972116,12
1,01-JAN-18 00:00:00,5966674,12
2,01-JAN-18 00:00:00,5959105,12
3,01-JAN-18 00:00:00,5966888,12
4,01-JAN-18 00:00:00,5965960,12


In [12]:
df_pro.shape

(116949113, 3)

In [16]:
filename = f'lines/PROGRNUMBER.csv'
df_pro.to_csv(filename, index_label=False)

As the data set is relatively large, storage is done first

In [ ]:
df_pro = pd.read_csv("lines/PROGRNUMBER.csv",low_memory=False)

### Get the maximum value of PROGRNUMBER, i.e. the total number of stops the bus has passed on the trip

In [13]:
df_pro_max=df_pro.groupby(['TRIPID','DAYOFSERVICE'], as_index=False)['PROGRNUMBER'].max()

In [14]:
df_pro_max

,TRIPID,DAYOFSERVICE,PROGRNUMBER
0,5955221,06-JAN-18 00:00:00,76
1,5955222,06-JAN-18 00:00:00,75
2,5955223,06-JAN-18 00:00:00,76
3,5955224,06-JAN-18 00:00:00,75
4,5955225,06-JAN-18 00:00:00,76
...,...,...,...
2182615,8592206,26-DEC-18 00:00:00,65
2182616,8592206,30-DEC-18 00:00:00,65
2182617,8592207,23-DEC-18 00:00:00,69
2182618,8592207,26-DEC-18 00:00:00,69


In [15]:
filename = f'lines/PROGRNUMBER_MAX.csv'
df_pro_max.to_csv(filename, index_label=False)